In [ ]:

# General imports
import glob
import shutil
import os
import sys
import traceback
from pathlib import Path
import pickle
import random
from enum import Enum
import datetime
from functools import partial
from multiprocess import Pool

from tqdm import tqdm
import easydict

# Interactive multithreading
from nbmultitask import ThreadWithLogAndControls
from time import sleep

# Imports relative to web mapping and displaying results
from ipyleaflet import Map, basemaps, ImageOverlay, TileLayer, GeoData
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from sidecar import Sidecar


# Array processing libraries
import numpy as np
from numpy import unravel_index
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile

# Geospatial data wrangling (mostly vector data)
import utm
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon,MultiPoint

# EO-learn/Sentinelhub imports
from eolearn.core import (
    EOWorkflow,
    Dependency,
    EOPatch,
    LoadFromDisk,
    SaveToDisk,
    OverwritePermission,
    FeatureType,
    LinearWorkflow,
    EOTask)
from eolearn.features import LinearInterpolation, SimpleFilterTask
from eolearn.geometry import PointSamplingTask, VectorToRaster
from eolearn.io import ExportToTiff, SentinelHubWCSInput, S2L1CWCSInput
from eolearn.mask import AddValidDataMaskTask, get_s2_pixel_cloud_detector, AddCloudMaskTask
from eolearn.ml_tools import MorphologicalOperations, MorphologicalStructFactory
from sentinelhub import CRS, BBoxSplitter, WcsRequest, DataSource, CustomUrlParam
from sentinelhub.constants import MimeType

# ML libraries
import lightgbm as lgb
from sklearn import metrics
from sklearn import preprocessing
import joblib

from eochallenge_prepare_data import (
    load_eopatch, 
    interpolate_eopatch, 
    intersect_aoi, 
    args, 
    train_model, 
    predict_eopatch, 
    plot_confusion_matrix
    )


In [ ]:

# The AOI should be in EPSG:4326
out_path = args.dest
bufsize = args.aoi_bufsize
row = args.zoom_level[0]
col = args.zoom_level[1]

aoi = gpd.read_file(args.aoi)
aoi['geometry'] = aoi['geometry'].to_crs(epsg=3857).buffer(bufsize).to_crs(epsg=4326)

country_list = ['Germany','Romania']
bbox_splitter_list = []
gdfs = [] # an array of GeoDataFrames to be plotted later
# Assuming there are two AOIs: one for Romania, one for Germany
for idx, aoi_el in zip(country_list, aoi.geometry):

    aoi_latlon = aoi_el.centroid.coords[0]
    utm_crs = utm.from_latlon(aoi_latlon[1], aoi_latlon[0])
    if aoi_latlon[1] > 0:
        if utm_crs[2] > 9:
            aoi_crs = 'EPSG:326%s' % utm_crs[2]
        else:
            aoi_crs = 'EPSG:3260%s' % utm_crs[2]
    else:
        if utm_crs[2] > 9:
            aoi_crs = 'EPSG:327%s' % utm_crs[2]
        else:
            aoi_crs = 'EPSG:3270%s' % utm_crs[2]

    # Split the AOI into a processing grid matching the OSM zoom level 12 grid

    aoi_temp = aoi['geometry'].to_crs(aoi_crs)

    print(country_list.index(idx))
    bbox_splitter = BBoxSplitter([aoi_temp[country_list.index(idx)]], aoi_crs, (row, col))
    bbox_splitter_list.append(bbox_splitter)

    bbox_list = bbox_splitter.get_bbox_list()
    print('Area bounding box: {}\n'.format(bbox_splitter.get_area_bbox().__repr__()))
    print('Each bounding box also has some info how it was created. Example:'
          '\nbbox: {}\ninfo: {}\n'.format(bbox_list[0].__repr__(), bbox_splitter.info_list[0]))
    print(f'\nThe AOI is covered by {len(bbox_splitter.bbox_list)} tiles to be processed.')

    # Create the path where to store the split processing grid.
    os.makedirs(f'{out_path}/aoi', exist_ok=True)

    with open(f'{out_path}/aoi/aoi_bbox_4326_{idx}_r{row}_c{col}.pkl', 'wb') as f:
        pickle.dump(bbox_splitter, f)

    geometry = [Polygon(bbox.get_polygon()) for bbox in bbox_list]
    idxs_x = [info['index_x'] for info in bbox_splitter.info_list]
    idxs_y = [info['index_y'] for info in bbox_splitter.info_list]

    df = pd.DataFrame({'index_x': idxs_x, 'index_y': idxs_y})
    gdf = gpd.GeoDataFrame(df, crs={'init': CRS.ogc_string(bbox_list[0].crs)}, geometry=geometry)
    gdf.head()

    gdf.to_file(f'{out_path}/aoi/aoi_bbox_4326_{idx}_r{row}_c{col}_{len(bbox_splitter.bbox_list)}.shp')
    gdfs.append(gdf)


In [ ]:

for aoi_idx, bbox_splitter in enumerate(bbox_splitter_list):
    if aoi_idx == 0:
        range_bbox = intersect_aoi(bbox_splitter, trainings[aoi_idx])
        range_idx = [bbox_splitter.bbox_list.index(bbox) for bbox in range_bbox]
        model, range_sample, labels_unique = train_model(range_idx, args.lulc_classes, args.time_range, out_path=f'{out_path}/{country_list[aoi_idx]}')
        for idx in [bbox_splitter.bbox_list.index(bbox) for bbox in bbox_splitter.bbox_list]:
            load_eopatch(bbox_splitter, time_range, training_arrays[aoi_idx],
            split_arrays[aoi_idx], training_vals[aoi_idx], f'{out_path}/{country_list[aoi_idx]}', row, col, idx)
            interpolate_eopatch(resample_range, training_vals[aoi_idx], f'{out_path}/{country_list[aoi_idx]}', idx)
            predict_eopatch(model, bbox_splitter, labels_unique,
                            args.proba, args.shap, f'{out_path}/{country_list[aoi_idx]}', idx)



In [ ]:
fig1 = plt.figure(figsize=(20,20))

class_names_test = [class_names[label] for label in np.unique(labels_test)]

plt.subplot(1,2,1)
conf_matrix_gbm = metrics.confusion_matrix(labels_test, plabels_test)
plot_confusion_matrix(conf_matrix_gbm,
                      classes=class_names_test,
                      normalize=True,
                      ylabel='Truth (RABA)',
                      xlabel='Predicted (GBM)',
                      title='Confusion matrix')
plt.xticks(rotation=90)

plt.subplot(1,2,2)
conf_matrix_gbm = metrics.confusion_matrix(plabels_test, labels_test)
plot_confusion_matrix(conf_matrix_gbm,
                      classes=class_names_test,
                      normalize=True,
                      xlabel='Truth (RABA)',
                      ylabel='Predicted (GBM)',
                      title='Transposed Confusion matrix')
plt.xticks(rotation=90)
plt.tight_layout()

fig2 = plt.figure(figsize=(20,5))


# In[ ]:


# Training Data Distribution Across Classes
label_ids, label_counts = np.unique(labels_train, return_counts=True)

plt.barh(range(len(label_ids)),label_counts)
plt.yticks(range(len(label_ids)), [class_names[i] for i in label_ids], fontsize=20)
plt.xticks(fontsize=20, rotation=45)
plt.title(f'Training Data Abundance Distribution across classes', fontsize=16, y=1.2)
plt.suptitle(f'Total sample size: {np.sum(label_counts)}',fontsize=14)


# In[ ]:


# Plot ROC Curve

#Calculate precision and recall rates.
class_labels = np.unique(np.hstack([labels_test,labels_train]))
print(class_names)

scores_test = model.predict_proba(features_test)
labels_binarized = preprocessing.label_binarize(labels_test, classes=class_labels)
plot_colors = ['xkcd:darkgreen', 'xkcd:lime','xkcd:tan', 'orange','xkcd:beige','crimson','xkcd:azure', 'xkcd:lavender','xkcd:lightblue'] #'white','xkcd:lavender', 'black'

fpr = dict()
tpr = dict()
roc_auc = dict()

for idx,lbl in enumerate(class_labels):
    fpr[idx], tpr[idx], _ = metrics.roc_curve(labels_binarized[:, idx], scores_test[:, idx])
    roc_auc[idx] = metrics.auc(fpr[idx], tpr[idx])

plt.figure(figsize=(20,10))

absent_value = []
for idx,lbl in enumerate(class_labels):
    if np.isnan(roc_auc[idx]):
        continue
    try:
        plt.plot(fpr[idx], tpr[idx], color=plot_colors[idx],lw=2, label=class_names[lbl]+' (%0.5f)' % roc_auc[idx])
    except:
        absent_value.append(lbl)
        continue

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 0.2])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title(f'Receiver operating characteristic for {project_name}', fontsize=20)
plt.legend(loc="lower right", prop={'size':15})
plt.show()


# In[ ]:


# Plot Feature Importance

# names of features
fnames = ['B2','B3','B4','B8','B11','B12','NDVI','NDWI','NORM']

# get feature importances and reshape them to dates and features
z = np.zeros(t1*f1)
z = model.feature_importances_
z = z.reshape((t1,f1))

fig3 = plt.figure(figsize=(20,18))
ax = plt.gca()

# plot the importances
im = ax.imshow(z,aspect=0.5)
plt.xticks(range(len(fnames)), fnames, rotation = 45, fontsize=20)
plt.yticks(range(t1), ['T{}'.format(i) for i in range(t1)], fontsize=20)

cax = fig3.add_axes([0.82, 0.125, 0.04, 0.755])
plt.colorbar(im, cax=cax)
plt.title(f'feature importance per date for {project_name}', fontsize=20, x=-7)
plt.show()


# In[ ]:


# Plot most important band/date combination

z_max = unravel_index(np.argmax(z), z.shape)
print(z_max)
z_min = unravel_index(np.argmin(z[z>0]), z.shape)
print(z_min)

bsub_tsub = np.swapaxes(np.array([eopatch.data['FEATURES_SAMPLED'] for eopatch in eopatches[range_sample]]), 1, 3)[...,z_min[0],z_min[1]].reshape(p1*h1*w1)
bopt_tsub = np.swapaxes(np.array([eopatch.data['FEATURES_SAMPLED'] for eopatch in eopatches[range_sample]]), 1, 3)[...,z_max[0],z_min[1]].reshape(p1*h1*w1)
bsub_topt = np.swapaxes(np.array([eopatch.data['FEATURES_SAMPLED'] for eopatch in eopatches[range_sample]]), 1, 3)[...,z_min[0],z_max[1]].reshape(p1*h1*w1)
bopt_topt = np.swapaxes(np.array([eopatch.data['FEATURES_SAMPLED'] for eopatch in eopatches[range_sample]]), 1, 3)[...,z_max[0],z_max[1]].reshape(p1*h1*w1)
labels = np.array([eopatch.mask_timeless['LULC_SAMPLED'] for eopatch in eopatches[range_sample]]).reshape(p1*h1*w1*1)

# remove nans
mask = np.any([np.isnan(bsub_tsub), np.isnan(bopt_tsub), np.isnan(bsub_topt), np.isnan(bopt_topt), labels==0],axis=0)
bsub_tsub, bopt_tsub, bsub_topt, bopt_topt, labels = [array[~mask] for array in [bsub_tsub, bopt_tsub, bsub_topt, bopt_topt, labels]]

fig4 = plt.figure(figsize=(20,20))

plot_labels = np.unique(labels)
plot_colors = lulc_cmap.colors
plot_colors = ['xkcd:darkgreen', 'xkcd:lime','xkcd:tan', 'orange','xkcd:beige','crimson','xkcd:azure', 'xkcd:lavender','xkcd:lightblue'] #'white','xkcd:lavender', 'black'
print(plot_labels)
print(plot_colors)

plt.subplot(2,2,1)
plt.hist([bsub_topt[labels == i] for i in plot_labels],100,(-0.4, 0.8),histtype='step',
         color=[plot_colors[i] for i in range(len(plot_labels))],
         label=[class_names[i] for i in plot_labels],
         )
plt.title(f'Most important band at least optimal date T{z_min[0]}', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(f'{fnames[z_max[1]]}', fontsize=20)
plt.legend(loc=1, prop={'size':15})

plt.subplot(2,2,2)
plt.hist([bopt_topt[labels == i] for i in plot_labels],100,(-0.4, 0.8),histtype='step',
         color=[plot_colors[i] for i in range(len(plot_labels))],
         )
plt.title(f'Most important band at most optimal date T{z_max[0]}', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(f'{fnames[z_max[1]]}', fontsize=20)

plt.subplot(2,2,3)
plt.title(f'Least important band at least optimal date T{z_min[0]}', fontsize=20)
plt.hist([bsub_tsub[labels == i] for i in plot_labels],100,(0.1, 0.7),histtype='step',
         color=[plot_colors[i] for i in range(len(plot_labels))],
         )
plt.xticks(fontsize=20)#     plt.yticks(fontsize=20)
plt.xlabel(f'{fnames[z_min[1]]}', fontsize=20)

plt.subplot(2,2,4)
plt.title(f'Least important band at most optimal date T{z_max[0]}', fontsize=20)
plt.hist([bopt_tsub[labels == i] for i in plot_labels],100,(0.1, 0.7),histtype='step',
         color=[plot_colors[i] for i in range(len(plot_labels))],
         )
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(f'{fnames[z_min[1]]}', fontsize=20)
plt.show()

pbar = tqdm(total=len(bbox_splitter.bbox_list))


# # Predict EOPatches using trained model
#

# # Plot a web map
# Some examples of implementation using ipywidget and ipyleaflet.
#

# In[ ]:


# Color mapping of the land cover classes to display.
newcolors = np.zeros((12,4))
newcolors[0,:] = np.asarray([0,0,0,0])
newcolors[1,:] = np.asarray([183, 219, 177, 255])
newcolors[2,:] = np.asarray([0,0,0,0])
newcolors[3,:] = np.asarray([149, 224, 145, 255])
newcolors[4,:] = np.asarray([255, 223, 145, 255])
newcolors[5,:] = np.asarray([0,0,0,0])
newcolors[6,:] = np.asarray([226, 193, 115, 255])
newcolors[7,:] = np.asarray([255, 244, 150, 255])
newcolors[8,:] = np.asarray([255, 150, 150, 255])
newcolors[9,:] = np.asarray([150, 171, 255, 255])
newcolors[10,:] = np.asarray([209, 183, 255, 255])
newcolors[11,:] = np.asarray([183, 255, 253, 255])

print(newcolors/255)
newcmp = ListedColormap(newcolors/255)
print(newcmp)


# In[ ]:


# Create a map using Stamen Terrain, centered on study area with set zoom level
m = Map(center=(37, -120), zoom=0, basemap=basemaps.Stamen.Toner)
m


# In[ ]:


sc = Sidecar(title='California Map')
with sc:
    display(m)


# In[ ]:


# Iteratively display the predicted tiles as they become available in folder structure
for tiff in glob.glob("/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/val_pred_sh_eopatch_*local.tiff"):
    out_path = "/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/folium/"+tiff[:-5].split('/')[-1]+"_4326.tiff"
    with rio.open(out_path) as src:
       boundary = src.bounds

    m.add_layer(ImageOverlay(url="http://172.29.254.183:8000/"+tiff[:-5].split('/')[-1]+"_4326.png", bounds=((boundary[1], boundary[0]),
                                            (boundary[3], boundary[2])), opacity=1))
    m


# In[ ]:


#Add a local tile server
os.chdir("/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/")
m.add_layer(TileLayer(url="http://172.29.254.183:8000/{z}/{x}/{y}.png"))
m


# In[ ]:


for tiff in glob.glob("/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/val_pred_sh_eopatch_*local.tiff"):
    # Create variables for destination coordinate system and the name of the projected raster
    src_crs = 'EPSG:32719'
    dst_crs = 'EPSG:4326'
    out_vrt = "/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/folium/"+tiff[:-5].split('/')[-1]+"_col1.tiff"
    out_vrt1 = "/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/folium/"+tiff[:-5].split('/')[-1]+"_col.tiff"
    out_path = "/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/folium/"+tiff[:-5].split('/')[-1]+"_4326.tiff"
    out_jpg = "/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/folium/"+tiff[:-5].split('/')[-1]+"_4326.png"
    out_tms = "/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/folium/"+tiff[:-5].split('/')[-1]+"_4326"
    #print(out_path)

    cmd = 'gdaldem color-relief %s /mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/col.txt %s -of Gtiff'%(tiff,out_vrt)
    get_ipython().system('{cmd}')
    """cmd1 = 'gdalwarp -of GTiff -overwrite -s_srs %s -t_srs %s %s %s'%(src_crs, dst_crs, out_vrt, out_vrt1)
    !{cmd1}
    os.remove(out_vrt)
    cmd2 = 'gdal_translate -of JPEG %s %s'%(out_vrt1,out_path)
    !{cmd2}"""

    #subprocess.call(["gdaldem","color-relief", tiff, "/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/col.txt",out_vrt,"-of" ,"VRT"])
    #subprocess.call(["gdalwarp","-of", "JPEG", "-s_srs",src_crs,"-t_srs",dst_crs, out_vrt, out_path])

    # Use rasterio package as rio to open and project the raster
    with rio.open(out_vrt) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Use rasterio package as rio to write out the new projected raster
        # Code uses loop to account for multi-band rasters
        with rio.open(out_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
    # Use rasterio to import the reprojected data as img
    with rio.open(out_path) as src:
        boundary = src.bounds
        #print(boundary)
        #img = src.read()
        #nodata = src.nodata

    cmd2 = "convert %s -transparent black -fuzz 11%% %s"%(out_path, out_jpg)
    get_ipython().system('{cmd2}')

    # Overlay raster called img using add_child() function (opacity and bounding box set)
    #m.add_child( folium.raster_layers.ImageOverlay(img[0], opacity=1, colormap=newcmp,
    #                                 bounds =[[boundary[1], boundary[0]], [boundary[3], boundary[2]]]))
    m.add_layer(ImageOverlay(url="http://172.29.254.183:8000/"+tiff[:-5].split('/')[-1]+"_4326.png", bounds=((boundary[1], boundary[0]),
                                             (boundary[3], boundary[2])), opacity=1))

    m

m.save(outfile= "/mnt/1t-drive/eopatch-L1C/nam_usa_uca/lulc_pred/folium/test.html")
